## Interview Assistance with ChatGPT
### This notebook is used to test APIs and to build a prove of concept

The soundcard library uses pulseaudio and must be used on your local machine.  

In [7]:
import soundcard as sc
import soundfile as sf
from datetime import datetime
import openai

openai.api_key = "YOUR_API_KEY"

The duration of the recording is a constant for testing. In pratical, this should be dynamic using a good implementation method.  
  
According to the Nyquist-Shannon theorem, the sample rate should be at least two times the highest frequency you intend to capture.  
By using the standard sample rate of 44.1 kHz, you’ll perfectly record frequencies slightly higher than 20,000 Hz, which is the highest frequency level humans can hear.  

In [2]:
# Set the duration of the recording in seconds
RECORD_SECONDS = 5
# Set the sample rate
SR = 44100

Loopback enables allows to record sounds produced by our system as if they were being redirected into the microphone.  
In other words, a virtual microphones that record the output of a speaker.

In [3]:
# get a list of all loopback devices (devices that record system audio):
lb_devices = sc.all_microphones(include_loopback=True)

In [4]:
#print out the avaiable loopback devices
for i in range(len(lb_devices)):
    try:
        print(f"{i}: {lb_devices[i].name}")
    except Exception as e:
        print(e)

0: MSI G241 (HD Audio Driver for Display Audio)
1: AI Noise-cancelling Output (ASUS Utility)
2: Headphones (Realtek(R) Audio)
3: Speakers (Realtek(R) Audio)
4: Microphone (Realtek(R) Audio)
5: Microphone Array (Realtek(R) Audio)
6: AI Noise-cancelling Input (ASUS Utility)


After testing several different loopback devices, some does not work as intended and will stop recording with no error message.  
Try choosing Headphones (Realtek(R) Audio).  

In [5]:
# select the correct loopback device
lb_device = lb_devices[2]

In [ ]:
# recording
print("Recording system audio...")
data = lb_device.record(numframes = RECORD_SECONDS * SR, samplerate= SR)
print("Done...")

In [ ]:
# Set the filename using the date time
now = datetime.now().strftime("%m-%d_%H-%M")
file_name = f"q_{now}.wav"
# write the recorded data to a WAV file
sf.write(file_name, data, SR)  

To open the audo file you must download ffmepg.  
download: https://ffmpeg.org/download.html  
how to: https://phoenixnap.com/kb/ffmpeg-windows

Other functionalities and implementations of the whisper API is worth reading.  
whisper github: https://github.com/openai/whisper  
  
sorce of sample interview question: https://www.youtube.com/watch?v=GJdiM-muYqc

In [8]:
# audio_file = open(file_name, "rb")
audio_file = open("sample_interview_question.wav", "rb")
# transcribe the auido file with whisper API
transcript = openai.Audio.translate("whisper-1", audio_file)
print(transcript["text"])

You're given a string and you're supposed to write a function that takes that string and returns the first Recurrent character in that string. So for example, if you're given this string as the input a b c a There's only one recurrent character in the string a so your function should return a If you're given this string b c a b a you see that there are two recurrent characters B and a and the first recurrent character is B So you should return B from your function If you're given this string ABC, there is no recurrent character in this string So your function should return either null or none depending on the language


Which system prompt to use to get better results. Or are there other ways to optimize the response from the models?

In [9]:
# prompt the model with the transcribed text and get the responce
responce = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are in a job or technical interview. Provide bullet point answers with a short explaination of reasoning."},
        {"role": "user", "content": transcript["text"]},
    ]
)
print(responce["choices"][0]["message"]["content"])

- Create a dictionary to store the count of each character in the string.
- Loop through the string and for each character check if it's already present in the dictionary.
- If it is, return that character as the first recurrent character.
- If not, add it as a key in the dictionary and initialize the count to 1.
- If the loop completes without finding a recurrent character, return null or none depending on the language.

Reasoning:
We can use a dictionary to efficiently keep track of the count of each character in the string. By storing the count of each character in the dictionary, we can easily check if a character is recurrent by just looking up its count in the dictionary. Looping through the string allows us to find the first recurrent character, since the first character that has a count greater than 1 is the first one that was repeated. If no recurrent character is found, we return null or none as specified by the question.


### Useful resources
Whisper API - Python and Gradio Tutorial: https://www.youtube.com/watch?v=Si0vFx_dJ5Y
